### Extract DEA Water Observation (WO) statistics and attach to shapefile of DEA waterbodies for specific area
##### DEA waterbodies dataset can be downloaded from: https://cmi.ga.gov.au/data-products/dea/693/dea-waterbodies-landsat#acces
##### This notebook is designed to be run on DEA Sandbox https://app.sandbox.dea.ga.gov.au/

In [3]:
import datacube
from datacube.utils import masking
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, 'Tools/')

from datacube.utils.cog import write_cog
from dea_tools.datahandling import load_ard
from datetime import datetime
from pytz import timezone
import geopandas as gpd
import gc
import time
import numpy as np
import rioxarray as riox

In [6]:
dc = datacube.Datacube(app='GDEs')

In [7]:
#set infile to shapefile used as input (eg waterbodies/GDEs) to attach WO statistics to
infile = "data/AquaticGDE_Dav.shp"
prefix = "DAV"

#read shape file into dataframe
gdf = gpd.read_file(infile)

#set coordinate reference system to be coordinate system of shapefile
crs = gdf.crs

#change projection to albers (to assist with buffering in metres)
gdf = gdf.to_crs("epsg:3577")
crs = gdf.crs

#set bounds to be extent of shapefile
bounds = gdf.bounds

#to ensure tiff will be outside all polygons, add 30m to bounds (had issue with poly being 2cm outside tiff)
minx = bounds.minx.min() -120   
maxx = bounds.maxx.max() +120
miny = bounds.miny.min() -120  
maxy = bounds.maxy.max() +120


In [8]:
#check coordinates are in Albers (m) and coordinate system is Albers (3577)
print (minx, miny, maxx, maxy)
print (crs)

109251.75764057248 -2411552.520632907 438235.1153589699 -2164764.5351402536
epsg:3577


In [9]:
# Extract the Water Observation (WO) collection 3 data
ds = dc.load(product='ga_ls_wo_fq_myear_3',
                  x=(minx, maxx),
                  y=(maxy, miny),
                  crs = 'EPSG:3577',
                  output_crs='EPSG:3577',
                  resolution=(-30, 30)
            )
ds

<xarray.Dataset>
Dimensions:      (time: 1, y: 8227, x: 10967)
Coordinates:
  * time         (time) datetime64[ns] 2004-07-01T23:59:59.999999
  * y            (y) float64 -2.165e+06 -2.165e+06 ... -2.412e+06 -2.412e+06
  * x            (x) float64 1.092e+05 1.093e+05 ... 4.382e+05 4.382e+05
    spatial_ref  int32 3577
Data variables:
    count_wet    (time, y, x) int16 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    count_clear  (time, y, x) int16 823 822 825 826 827 ... 873 874 873 872 873
    frequency    (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [6]:
# save the raster as geotiff
write_cog(geo_im= ds.frequency, fname='data/{}_WO_fq_myear_c3.tif'.format(prefix), overwrite=True)


PosixPath('data/DAV_WO_fq_myear_c3.tif')

In [11]:
# open WOfS raster and add to dictionary
wo_fq_myear_raster = riox.open_rasterio("data/{}_WO_fq_myear_c3.tif".format(prefix))

# create array of floats for writing the wofs filtered summary
arrays = {"wo_fq_myear_mean": np.nan * np.ones(shape = len(gdf), dtype = float),
         "wo_fq_myear_median": np.nan * np.ones(shape = len(gdf), dtype = float),
         "wo_fq_myear_min": np.nan * np.ones(shape = len(gdf), dtype = float),
         "wo_fq_myear_max": np.nan * np.ones(shape = len(gdf), dtype = float),}           

In [8]:
#check output filename is correct
print ("data/{}_WO_fq_myear_c3.tif".format(prefix))

data/DAV_WO_fq_myear_c3.tif


In [47]:
#initialise time so can calculate how long process takes
start_time = time.time()

#datetime.now() gets current time; astimezone is function to convert to timezone; timezone('Australia/Sydney') is the timezone to conver to
now = datetime.now().astimezone(timezone('Australia/Sydney'))  #setting time to be Sydney timezone
timef = now.strftime("%d-%m-%Y %H:%M:%S")  #formatting the time
print (timef)
# iterate through the geodataframe and get the geometry
for i, (index, row) in enumerate(gdf.iterrows()):
    if i%100 == 0:
        print(i)
    poly = row.geometry
    ######################################################################################################
    #CHECK ANY POLYS THAT GET PULLED OUT FOR BEING TOO SMALL AND DELETE FROM SHAPEFILE IF NOT NEEDED
    #> 800m seems to grab values that are more than half the pixel, so those < 800m should be ok to exclude
    #######################################################################################################
    if row.geometry.area > 800: 
        clipped_raster = wo_fq_myear_raster.rio.clip([poly])
    # extract array
        arr = clipped_raster.values[0]
        
        # Flag polygons with all No Data, probably due to shadows in Water Observations dataset
        if np.all(np.isnan(arr)):
            print("Polygon with ufi {} contains only NaN data, likely due to known shadows in Water Observations dataset".format(row["ufi"]))
    
    # remove null values and save the mean to the array
        arrays["wo_fq_myear_mean"][i] = np.nanmean(arr[arr != clipped_raster._FillValue])
        arrays["wo_fq_myear_median"][i] = np.nanmedian(arr[arr != clipped_raster._FillValue])
        arrays["wo_fq_myear_max"][i] = np.nanmax(arr[arr != clipped_raster._FillValue])
        arrays["wo_fq_myear_min"][i] = np.nanmin(arr[arr != clipped_raster._FillValue])
    else:
        #print UFI of polygons in shapefile that are too small to keep track of small polys in case they are needed, 
        #in which case reduce geometry.area in if statement above be a value < 800
        print("Polygon too small to process \n","UFI",row.ufi) #
        continue
print("Time elapsed {} seconds".format(time.time() - start_time))


27-07-2023 11:24:42
0
Polygon too small to process 
 UFI 4382571
Polygon too small to process 
 UFI 4384104
Polygon too small to process 
 UFI 4381499
Polygon too small to process 
 UFI 4379490
100
Polygon too small to process 
 UFI 4379495
Polygon too small to process 
 UFI 4379521
200
Polygon too small to process 
 UFI 3586946
Polygon too small to process 
 UFI 4379508
Polygon with uif 4383280 has only NaN data


/tmp/ipykernel_188/582731952.py:25: RuntimeWarning: Mean of empty slice
  arrays["wo_fq_myear_mean"][i] = np.nanmean(arr[arr != clipped_raster._FillValue])
/env/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/tmp/ipykernel_188/582731952.py:27: RuntimeWarning: All-NaN slice encountered
  arrays["wo_fq_myear_max"][i] = np.nanmax(arr[arr != clipped_raster._FillValue])
/tmp/ipykernel_188/582731952.py:28: RuntimeWarning: All-NaN slice encountered
  arrays["wo_fq_myear_min"][i] = np.nanmin(arr[arr != clipped_raster._FillValue])


Polygon too small to process 
 UFI 4381498
300
Polygon too small to process 
 UFI 4379527
Polygon too small to process 
 UFI 4382561
Polygon too small to process 
 UFI 4379504
Polygon too small to process 
 UFI 4379562
400
Polygon too small to process 
 UFI 4379505
Polygon too small to process 
 UFI 4381497
Polygon too small to process 
 UFI 4384106
500
Polygon too small to process 
 UFI 4379494
Polygon too small to process 
 UFI 4381463
Polygon too small to process 
 UFI 4382568
600
Polygon too small to process 
 UFI 4379477
Polygon too small to process 
 UFI 4379525
Polygon too small to process 
 UFI 4379507
Polygon too small to process 
 UFI 4379500
Polygon too small to process 
 UFI 3586036
Polygon too small to process 
 UFI 3586033
Polygon too small to process 
 UFI 4379528
Polygon too small to process 
 UFI 4379506
Polygon too small to process 
 UFI 3586035
700
Polygon too small to process 
 UFI 4379510
Polygon too small to process 
 UFI 3586034
Polygon too small to process 
 UFI

In [54]:
# add the arrays to the geodataframe
gdf['wo_mean'] = arrays['wo_fq_myear_mean']
gdf['wo_p50'] = arrays['wo_fq_myear_median']
gdf['wo_min'] = arrays['wo_fq_myear_min']
gdf['wo_max'] = arrays['wo_fq_myear_max']
#gdf['tcw_p50'] = arrays['TCW_PC_50']


In [55]:
# save the geodataframe to a file
#to automatically create filename with same name as input file but with 'stat's added, need to strip .shp from name
filename = (infile.rstrip(".shp"))
filename = (filename + '_WO_fq_myear_c3.shp')
###not sure if qbove will work as has 'r' below before path in quotes

gdf.to_file(filename, index =False)
#original code to write file (use if can't get above working)
#gdf.to_file(r"data/UD_Waterbodies_intersect_aquaticGDEs_TCW.shp", index =False)